# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('Orders.csv')
orders

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here
grouped_customerID = pd.pivot_table(orders, values='amount_spent', index=['CustomerID'], aggfunc=np.sum)
grouped_customerID

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [9]:
grouped_customerID.reset_index(inplace=True)
grouped_customerID.sort_values(by=['amount_spent'], axis=0, ascending=False, inplace=True)
grouped_customerID.head()

,CustomerID,amount_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79
3,16446,168472.50
4,14911,143825.06


In [5]:
grouped_customerID[['amount_spent']].describe()

,amount_spent
count,4339.000000
mean,2053.793018
std,8988.248381
min,0.000000
25%,307.245000
50%,674.450000
75%,1661.640000
max,280206.020000


In [6]:
q95 = grouped_customerID['amount_spent'].quantile(0.95) 
q95
#thanks to nonunicorn on GitHiub

5840.181999999982

In [8]:
q75 = grouped_customerID['amount_spent'].quantile(0.75) 
q75

1661.64

In [22]:
grouped_customerID['customer_type'] = [None] * len(grouped_customerID)
grouped_customerID.head()

,CustomerID,amount_spent,customer_type
0,14646,280206.02,None
1,18102,259657.30,None
2,17450,194550.79,None
3,16446,168472.50,None
4,14911,143825.06,None


In [26]:
def assign_customer_type(amount):
    if amount >= q95:
        return 'VIP'
    elif q75 <= amount < q95:
        return 'Preferred'
    else:
        return 'Other'

In [28]:
grouped_customerID['customer_type'] = grouped_customerID['amount_spent'].apply(assign_customer_type)
grouped_customerID

,CustomerID,amount_spent,customer_type
0,14646,280206.02,VIP
1,18102,259657.30,VIP
2,17450,194550.79,VIP
3,16446,168472.50,VIP
4,14911,143825.06,VIP
...,...,...,...
4334,17956,12.75,Other
4335,16454,6.90,Other
4336,14792,6.20,Other
4337,16738,3.75,Other


In [31]:
customer_type_mapping = grouped_customerID.set_index('CustomerID')['customer_type'].to_dict()
customer_type_mapping

{14646: 'VIP',
 18102: 'VIP',
 17450: 'VIP',
 16446: 'VIP',
 14911: 'VIP',
 12415: 'VIP',
 14156: 'VIP',
 17511: 'VIP',
 16029: 'VIP',
 12346: 'VIP',
 16684: 'VIP',
 14096: 'VIP',
 13694: 'VIP',
 15311: 'VIP',
 13089: 'VIP',
 17949: 'VIP',
 15769: 'VIP',
 15061: 'VIP',
 14298: 'VIP',
 14088: 'VIP',
 15749: 'VIP',
 12931: 'VIP',
 17841: 'VIP',
 15098: 'VIP',
 13798: 'VIP',
 16013: 'VIP',
 16422: 'VIP',
 12748: 'VIP',
 15838: 'VIP',
 17404: 'VIP',
 17389: 'VIP',
 13098: 'VIP',
 14680: 'VIP',
 13081: 'VIP',
 13408: 'VIP',
 17857: 'VIP',
 16333: 'VIP',
 13777: 'VIP',
 12753: 'VIP',
 12744: 'VIP',
 16210: 'VIP',
 17675: 'VIP',
 17381: 'VIP',
 15039: 'VIP',
 12471: 'VIP',
 12731: 'VIP',
 15159: 'VIP',
 12901: 'VIP',
 12678: 'VIP',
 14031: 'VIP',
 17428: 'VIP',
 13767: 'VIP',
 13881: 'VIP',
 16839: 'VIP',
 12921: 'VIP',
 14607: 'VIP',
 15856: 'VIP',
 17677: 'VIP',
 15189: 'VIP',
 14051: 'VIP',
 15513: 'VIP',
 16133: 'VIP',
 14866: 'VIP',
 16705: 'VIP',
 12681: 'VIP',
 12621: 'VIP',
 12540: 'V

In [33]:
orders['Customer Type'] = orders.loc[:, 'CustomerID'] 
orders['Customer Type'] = orders['Customer Type'].map(customer_type_mapping)
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Customer Type
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [51]:
# your code here

gr_ctype_country = orders.groupby(['Country', 'Customer Type']).size().unstack(fill_value=0)
gr_ctype_country

Customer Type,Other,Preferred,VIP
Country,,,
Australia,157,130,898
Austria,240,158,0
Bahrain,17,0,0
Belgium,474,1503,54
Brazil,32,0,0
Canada,16,135,0
Channel Islands,159,225,364
Cyprus,163,203,248
Czech Republic,25,0,0


In [53]:
top_vip_country = gr_ctype_country['VIP'].idxmax()
print(f"Country with the most VIP customers: {top_vip_country}")

Country with the most VIP customers: United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [54]:
# your code here

top_vip_preferred_country = gr_ctype_country[['VIP', 'Preferred']].sum(axis=1).idxmax()
print(f"Country with the most VIP + Preferred customers combined: {top_vip_preferred_country}")

Country with the most VIP + Preferred customers combined: United Kingdom
